In [3]:
import sys
import math
import numpy as np
from collections import deque
from tqdm import tqdm
import pygame
import json
import os.path
import matplotlib.pyplot as plt
import random 


Paramcar={"carheight":5,"carlength":5,"xcenterpos" :2.5,"ycenterpos":50,"place_in_road":1,"car_frequency":0.1,"v0":30 }
Paramtruck={"carheight":5,"carlength":10,"xcenterpos" :5,"ycenterpos":50,"place_in_road":1,"car_frequency":0.1,"v0":20 }
Paramdrivertruck={"maxspeed":90,"delta":5,"jamd":5,"maxa":2,"react_time":2.5,"breaking":1}
Paramdriver={"maxspeed":120,"delta":4,"jamd":1,"maxa":4,"react_time":2.5,"breaking":1.5}
Paramroad={"car_frequency":1,"position":0,"lastadding_t":0,"lengthroad":1000,"roadwidth":10,"yposition":50,"maxnumbercar":30}
Carortruckpercentage=0.8
delta_t = 0.3
runtime=1000
class Car:
    def __init__(self,Paramcar=Paramcar,Paramdriver=Paramdriver):
        self.v=Paramcar["v0"]
        self.carheight=Paramcar["carheight"]
        self.carlength=Paramcar["carlength"]
        self.xcenterpos=Paramcar["xcenterpos"]
        self.ycenterpos=Paramcar["ycenterpos"]
        self.place_in_road=Paramcar["place_in_road"]
        self.car_frequency=Paramcar["car_frequency"]
        self.posback=Paramcar["xcenterpos"]-Paramcar["carlength"]/2
        self.posfront=Paramcar["xcenterpos"]+Paramcar["carlength"]/2
        
        #for the intelligent driving model
        self.jamd=Paramdriver["jamd"]
        self.maxspeed=Paramdriver["maxspeed"]
        self.delta=Paramdriver["delta"]
        self.maxa=Paramdriver["maxa"]
        self.react_time=Paramdriver["react_time"]
        self.breaking=Paramdriver["breaking"]
        
        #local variable used as buffer
        self.localpos = Paramcar["xcenterpos"]
        self.localv = Paramcar["v0"]
        self.localaccel = 0.
        
    def intelligentdriving(self,other_v,s):
        v=self.v
        delta_v = v-other_v
        s_star = (self.jamd + self.react_time*v + (v + delta_v) / (2 * math.sqrt(self.maxa * self.breaking)))
        accel = (self.maxa * (1 - math.pow(v/self.maxspeed, self.delta) - math.pow(s_star/s, 2)))
        return accel
    
    def serialize(self):
        """Serialize the car

        Returns:
            Dict with:
                pos: Position of the car
                v: Speed of the car
                accel: Acceleration of the car
                length: Length of the car
        """
        return {'position': self.xcenterpos, 'v': self.v, 'accel': self.localaccel, 'length': self.carlength,'height':self.carheight}

  
       
class Road:
    def __init__(self,Paramroad=Paramroad):
        self.car_frequency = Paramroad["car_frequency"]
        self.position = Paramroad["position"]
        self.lengthroad = Paramroad["lengthroad"]
        self.carlist = [] # List of cars on the road
        self.lastadding_t=Paramroad["lastadding_t"]
        self.yposition=Paramroad["yposition"]
        self.roadwith=Paramroad["roadwidth"]
        self.maxnumbercar=Paramroad["maxnumbercar"]
    def sort_carlist_bypos(self):
        self.carlist = sorted(self.carlist, key=lambda car: car.posback)
    def carfront(self,car):
        lenlist=len(self.carlist)
        carindex=self.carlist.index(car)
        if lenlist==0  :
            carfront=None
        else :
            carfront = self.carlist[(carindex+1)%lenlist]
        if carindex==lenlist-1:
            carislast=True
        else :
            carislast=False
        return carfront,carislast
    
    def addcar(self):
        lenlist=len(self.carlist)
        rand = random.random()
        
        
        if lenlist==0:
            self.carlist.append(Car())
            return True
        else:
            posmin=self.carlist[0].posback
            posmax=self.carlist[lenlist-1].posfront
            if rand >Carortruckpercentage:
                if (posmin > Paramcar["carlength"]) & (posmax < (self.lengthroad-2*Paramcar["carlength"])):
                    for car in self.carlist:
                        car.place_in_road +=1
                    self.carlist.insert(0,Car())
                    return True
                else:
                    return False
            else :
                if (posmin > Paramtruck["carlength"]) & (posmax < self.lengthroad-2*Paramcar["carlength"]):
                    for car in self.carlist:
                        car.place_in_road +=1
                    self.carlist.insert(0,Car(Paramcar=Paramtruck,Paramdriver=Paramdrivertruck))
                    return True
                else:
                    return False
    def update_local(self,car, delta_t: float):
        
        car_front, carislast = self.carfront(car)
        # Update local speed
        if carislast==False:
            s = abs(car_front.posback - car.posfront) if car_front is not None else self.lengthroad
        else :
            s=abs(car_front.posback+(self.lengthroad-car.posfront)) if car_front is not None else self.lengthroad
        s = max(0.000000001, s) # s can't be 0 or it will break things so we make s small
        other_v = car_front.v if car_front is not None else car.v
        car.localpos=(car.xcenterpos + car.v*delta_t)%self.lengthroad
        car.localv = car.v
        car.localaccel = car.intelligentdriving(other_v=other_v, s=s) 
        car.localv += car.localaccel*delta_t
        car.localv = max(car.localv, 0)
        
    def update_global(self,car):
        """
        Update global state
        """

        car.xcenterpos = car.localpos
        car.posback = car.xcenterpos - car.carlength / 2
        car.posfront = car.xcenterpos + car.carlength / 2

        car.v = car.localv
        
    def updateall(self, delta_t: float):
        """Create cars and update all the cars in the list
        Args:
            delta_t: Time step to simulate

        Returns: True if a car reached the end of the road during this step
        """

        #car_reached_end = False

        # Update the local state of all the cars
        for car in self.carlist:
            self.update_local(car,delta_t)

        # Update the global state of all the cars
        for car in self.carlist:
            self.update_global(car)
        
        self.sort_carlist_bypos()
        
            # If a car is outside the road, then delete it from the list and set the return flag
           # if car.posback > self.lengthroad:
               # self.carlist.remove(car)
               # car_reached_end = True

        # Try to spawn a car if the time has come
        self.lastadding_t += delta_t
        isadded=False
        lenlist=len(self.carlist)
        if (self.lastadding_t>=1/self.car_frequency) & (lenlist<self.maxnumbercar) :
            isadded=self.addcar()
        if isadded==True:
            self.lastadding_t = 0

        return True            
    
    

#############################################################################################
#############################################################################################


def show_pygame(data, delta_t):
    import pygame
    import pygame.gfxdraw
    
    SPEED = 1 # playback speed of the data (1 for real time)

    SIZE = WIDTH, HEIGHT = Paramroad["lengthroad"], 100
    SCREEN = pygame.display.set_mode(SIZE)
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)

    CLOCK = pygame.time.Clock()

    pygame.init()
    
    
    for datapoint in data:
        for event in pygame.event.get():
            if event.type == pygame.QUIT: 
                pygame.quit()
                print("vous avez quitté la fenêtre pygame")
                return 
            if event.type == pygame.KEYDOWN:
                # These two change the playback speed with the arrow keys
                if event.key == pygame.K_UP: SPEED += 1
                if event.key == pygame.K_DOWN: SPEED = max(1, SPEED - 1)

        SCREEN.fill(BLACK)
        pygame.gfxdraw.hline(SCREEN, 0, WIDTH, Paramroad["yposition"]-int(Paramroad["roadwidth"]/2), WHITE)
        pygame.gfxdraw.hline(SCREEN, 0, WIDTH, Paramroad["yposition"]+int(Paramroad["roadwidth"]/2), WHITE)
        for car in datapoint:
            #car_surface = pygame.Surface((car['length'], car.carheight)) # Make a pygame surface for the car
            #car_surface.fill(WHITE)                          # Fill the surface with white
            #car_rect = car_surface.get_rect()                # Get the rect (bounding area)
            #car_rect.center = (car['position'],50)                  # Set center of the rect to position of the car
            #SCREEN.blit(car_surface, car_rect)               # Draw the car on the screen
            
            rectcar=pygame.Rect(car['position']-car['length']/2,Paramroad["yposition"]-car['height']/2,car['length'],car['height'])
            pygame.gfxdraw.box(SCREEN,rectcar,WHITE)
            
            
            
        pygame.display.update()
        pygame.display.flip()# Update the screen
        CLOCK.tick(1/delta_t * SPEED)                       # Tick the clock according to the playback speed
    pygame.quit()
    print("la simulation à atteind la limite de temps")

class Simulation:
    """Class to manage the simulation

    Args:
        params_list: List of car params defining the different car types
        road_position: Position of the top most lane
        road_length: Road length
        road_lanes: Amount of lanes
        road_lane_width: Width of the car lanes
        delta_t: Time step to use when running the simulation
    """

    def __init__(self, delta_t=0.5):
        self.delta_t = delta_t
        self.road = Road()
        self.end = False # Flag to end the simulation

    def step(self):
        """Run just one step of the simulation

        Returns: True if a car has reached the end of the simulation
        """

        self.end |= self.road.updateall(delta_t=self.delta_t)
        return self.road.carlist

    def run(self, time=None):
        """Run the simulation either for `time` seconds or until a car reaches the end of the road
        
        Args:
            optional time: Amount of time (in seconds) to run the simulation
        """

        data = deque()
        if time is not None:
            numbercar=[]
            # This runs the simulation for time seconds
            for t in tqdm(np.arange(0, time, self.delta_t)):
                data.append([car.serialize() for car in self.step()])
                numbercar.append(len(self.step()))
        else:
            # This runs the simulation until a car reaches the end
            with tqdm(total=self.road.lengthroad) as pbar:
                lastpos = 0
                while not self.end:
                    first_car = None
                    cars_step = []
                    for car in self.step():
                        cars_step.append(car.serialize())
                        first_car = car if first_car is None or car.xcenterpos > first_car.xcenterpos else first_car
                    data.append(cars_step)
                    if first_car is not None:
                        pbar.set_description("#cars: " + str(len(self.road.carlist)))
                        pbar.update(max(int(first_car.xcenterpos) - lastpos, 0))
                        lastpos = int(first_car.xcenterpos)
        return list(data),numbercar       
        
     # Time step of the simulation
sim = Simulation(delta_t=delta_t)
data,numb = sim.run(runtime)
show_pygame(data,delta_t)


def make_filename(f):
    fsplit = f.rsplit(".", 1)
    name = fsplit[0]
    ext = fsplit[1] if len(fsplit) > 1 else ""
    suff = ""
    i = 1
    while os.path.isfile(f'{name}{suff}.{ext}'): suff = i ; i += 1
    return f'{name}{suff}.{ext}'


def make_dots(car_data: list, time_div: float, delta_x: float, colors: list,road_length=Paramroad["lengthroad"]):

    """Make car dot plot

        Args:
            carData: List with cars for each time step (generated by the simulation)
            roadLength: Length of the road in the simulation
            time_div: Time resampling factor of the data points (use 1 every i-th datapoint)
            delta_x: Minimum change in distance (how much distance a pixel represents)
            colors: Colors to use for each lane
     """

    print("Generating graph...")
    pixel_plot = np.full((len(car_data[::time_div]), round(road_length/delta_x), 3), (255., 255., 255.))
    for i, data_t in enumerate(tqdm(car_data[::time_div])):
        for car in data_t:
            x = int(np.floor(car['pos'][0]/delta_x))
            if x >= pixel_plot.shape[1]:  continue
            pixel_plot[i][x] *= colors[int(car['pos'][1]/5)]
            pixel_plot[i][x] = pixel_plot[i][x] * (255 / max(0.00001, pixel_plot[i][x].max()))

    return pixel_plot

def make_dots_bw(car_data: list, time_div: float, delta_x: float,road_length=Paramroad["lengthroad"]):

        """Make car dot plot in black and white only

        Args:
            carData: List with cars for each time step (generated by the simulation)
            roadLength: Length of the road in the simulation
            time_div: Time resampling factor of the data points (use 1 every i-th datapoint)
            delta_x: Minimum change in distance (how much distance a pixel represents)
        """

        print("Generating graph...")
        pixel_plot = np.full((len(car_data[::time_div]), round(road_length/delta_x)), (255.))
        for i, data_t in enumerate(tqdm(car_data[::time_div])):
            for car in data_t:
                x = int(np.floor(car['position']/delta_x))
                if x >= pixel_plot.shape[1]:  continue
                pixel_plot[i][x] -= 255./2
                pixel_plot[i][x] = max(pixel_plot[i][x], 0)

        return pixel_plot
    
def dots_to_image(pixel_plot, filename, mode='L', overwrite=False):
    from PIL import Image
    filename = (make_filename(filename) if not overwrite else filename)
    
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename), exist_ok=True)
    img = Image.fromarray(pixel_plot.astype('uint8'), mode=mode)
    img.save(filename)


dots = make_dots_bw(data, time_div=1, delta_x=10)      # Make the dot graph
dots_to_image(dots, f'Documents/Doctestcrossing/allin_maxspeed{Paramdriver["maxspeed"]}_maxa{Paramdriver["maxa"]}_breaking{Paramdriver["breaking"]}_time{runtime}_roadlength{Paramroad["lengthroad"]}.png', overwrite=False)                         # Save the dot graph
#dots_to_image(dots, "Documents/filename_graph.png", overwrite=True)                         # Save the dot graph

print(numb[0])

100%|██████████| 3334/3334 [00:01<00:00, 2057.39it/s]


vous avez quitté la fenêtre pygame
Generating graph...


100%|██████████| 3334/3334 [00:00<00:00, 7337.19it/s]


0


In [2]:
show_pygame(data,delta_t)

vous avez quitté la fenêtre pygame


In [3]:


def tri_et_retient_ordre(liste):
    """
    Trie une liste donnée et retourne la liste triée et une liste d'indices qui représente l'ordre original des éléments.
    """
    indices = list(range(len(liste)))
    indices.sort(key=lambda i: liste[i])
    liste_triee = [liste[i] for i in indices]
    return liste_triee, indices
l=[5,78,45,695,2,35,12,]
a,b = tri_et_retient_ordre(l)
print(a,b)

[2, 5, 12, 35, 45, 78, 695] [4, 0, 6, 5, 2, 1, 3]


In [5]:
def addcar(self):
        new_car = Car()
        if len(self.carlist)==0:
            self.carlist.append(new_car)
            return True
        else:
            posmin=self.lengthroad
            indexmin=0
            posmax=0
            indexmax=0
            for car in self.carlist:
                if car.xcenterpos<posmin :
                    posmin=car.xcenterpos
                    indexmin=self.carlist.index(car)
                if car.xcenterpos>posmax :
                    posmax=car.xcenterpos
                    indexmax=self.carlist.index(car)
            if (posmin > 50) & (posmax < self.lengthroad-50):
                for car in self.carlist:
                    car.place_in_road +=1
                self.carlist.insert(0,new_car)
                return True
            else:
                return False
            
        #sortedpos=[]  a mettre dans les varibles de road
        #sortedindex=[]
    def tri_et_retient_ordre(self):
    """
    Trie une liste donnée et retourne la liste triée et une liste d'indices qui représente l'ordre original des éléments.
    """
    listpos=[]
    for car in self.carlist:
        listpos.append(car.xcenterpos)
        
    indices = list(range(len(listpos)))
    indices.sort(key=lambda i: listpos[i])
    sortedlist = [listpos[i] for i in indices]
    return sortedpos, indices

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 28)